In [2]:
import os
os.chdir('..')

In [86]:
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [31]:
df = pd.read_csv('data/player-contracts.csv', usecols=('Player', 'Tm', '2020-21'))
df.columns = ('Player', 'Team', 'Salary')
df.Player = df.Player.apply(lambda x: x.split('\\')[0])  # removes hyperlink in name
print(df.shape)
df.head()

(481, 3)


,Player,Team,Salary
0,Stephen Curry,GSW,43006362.0
1,Russell Westbrook,WAS,41358814.0
2,Chris Paul,PHO,41358814.0
3,John Wall,HOU,41254920.0
4,James Harden,BRK,40824000.0


In [49]:
def get_name(row):
    if pd.isna(row['Eastern Conference']):
        return row['Western Conference']
    return row['Eastern Conference']

In [76]:
confs = pd.concat((pd.read_csv('data/east-conf.csv'), pd.read_csv('data/west-conf.csv')))
confs['Team'] = confs.apply(lambda row: f"{row['Eastern Conference']} {row['Western Conference']}", axis=1)
confs.head()

,Eastern Conference,W,L,W/L%,GB,PS/G,PA/G,SRS,Western Conference,Team
0,Philadelphia 76ers (1),20.0,11.0,0.645,—,114.1,111.4,2.12,NaN,Philadelphia 76ers (1) nan
1,Brooklyn Nets (2),20.0,12.0,0.625,0.5,121.0,117.0,3.82,NaN,Brooklyn Nets (2) nan
2,Milwaukee Bucks (3),18.0,13.0,0.581,2,119.2,112.1,6.61,NaN,Milwaukee Bucks (3) nan
3,Indiana Pacers (4),15.0,14.0,0.517,4,113.4,111.9,1.61,NaN,Indiana Pacers (4) nan
4,Toronto Raptors (5),16.0,15.0,0.516,4,113.1,110.3,2.42,NaN,Toronto Raptors (5) nan


In [59]:
abb = pd.read_csv('data/abbreviations.csv')
abb.columns = ('Abb', 'Full')
abb.head()

,Abb,Full
0,ATL,Atlanta Hawks
1,BKN,Brooklyn Nets
2,BOS,Boston Celtics
3,CHA,Charlotte Hornets
4,CHI,Chicago Bulls


In [64]:
new_df = pd.merge(df, abb, left_on='Team', right_on='Abb')
new_df.head()

,Player,Team,Salary,Abb,Full
0,Stephen Curry,GSW,43006362.0,GSW,Golden State Warriors
1,Klay Thompson,GSW,35361360.0,GSW,Golden State Warriors
2,Andrew Wiggins,GSW,29542010.0,GSW,Golden State Warriors
3,Draymond Green,GSW,22246956.0,GSW,Golden State Warriors
4,Kelly Oubre Jr.,GSW,14375000.0,GSW,Golden State Warriors


In [73]:
new_new_df = new_df.groupby('Full')['Salary'].sum().reset_index()
new_new_df.head()

,Full,Salary
0,Atlanta Hawks,115248809.0
1,Boston Celtics,117111401.0
2,Chicago Bulls,124006443.0
3,Cleveland Cavaliers,130303002.0
4,Dallas Mavericks,126463122.0


In [83]:
new_new_df['Wins'] = ''
for i, row1 in new_new_df.iterrows():
    for j, row2 in confs.iterrows():
        if row1['Full'] in row2['Team']:
            new_new_df.loc[i, 'Wins'] = row2.W

In [87]:
new_new_df['$ per win'] = new_new_df.apply(lambda row: row.Salary / row.Wins, axis=1)
new_new_df.head()

,Full,Salary,Wins,$ per win
0,Atlanta Hawks,115248809.00000,13.0,8865293.00000
1,Boston Celtics,117111401.00000,15.0,7807426.73333
2,Chicago Bulls,124006443.00000,13.0,9538957.15385
3,Cleveland Cavaliers,130303002.00000,10.0,13030300.20000
4,Dallas Mavericks,126463122.00000,13.0,9727932.46154


In [88]:
new_new_df

,Full,Salary,Wins,$ per win
0,Atlanta Hawks,115248809.00000,13.0,8865293.00000
1,Boston Celtics,117111401.00000,15.0,7807426.73333
2,Chicago Bulls,124006443.00000,13.0,9538957.15385
3,Cleveland Cavaliers,130303002.00000,10.0,13030300.20000
4,Dallas Mavericks,126463122.00000,13.0,9727932.46154
5,Denver Nuggets,125885711.00000,16.0,7867856.93750
6,Detroit Pistons,116186342.00000,8.0,14523292.75000
7,Golden State Warriors,173524090.00000,16.0,10845255.62500
8,Houston Rockets,131263516.00000,11.0,11933046.90909
9,Indiana Pacers,125450042.00000,15.0,8363336.13333
